# Automated Feature Engineering

In this notebook, we will implement an automated feature engineering solution in Featuretools for a set of online retail purchases. The dataset (available from the [UCI machine learning repository](https://archive.ics.uci.edu/ml/datasets/online+retail#)), a series of time-stamped purchases, does not come with any labels, so we'll have to make our own prediction problem. After defining the problem, we can use automated feature engineering to build a set of features that we then feed to a machine learning model for training. 

This set of data is great practice both for defining our own prediction problem, and for using some of the time-based capabilities of Featuretools such as cutoff times. Whenever we have time-series data, we need to be extra careful to not "leak labels" or use information from the future to predict a past event. Usually, when we're doing manual feature engineering, this can be quite an issue and often, a system will work well in development but utterly fail when deployed because it was trained on illegitimate data. Fortunately, Featuretools will take care of the time issue for us, creating a rich set of features that follow any time limitations. 

## Roadmap 

Following is an outline for this notebook:

1. Read in data, inspect, and clean
2. Develop a prediction problem
    * Create a dataframe of labels - what we want to predict, and cutoff times - the point that all data must come before for predicting a label
3. Create an entityset and add entities
    * Normalize the original table to develop new tables 
    * These new tables can be used for making features
3. Run deep feature sythesis on the entityset to make features
    * Use the cutoff times to make features using valid data for each label
4. Use the features to train a machine learning model
    * Measure performance of the model relative to an informed baseline
5. Tune deep feature synthesis 
    * Specify custom primitives
    * Adjust maximum depth of features
6. Use random search for hyperparameter tuning the model

This problem is a great display of both the time and feature-creation capabilities of Featuretools. Doing this problem by hand and ensuring we use only valid data for each label is a daunting task! 

In [1]:
# Data manipulation
import pandas as pd 
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Automated feature engineering
import featuretools as ft

# Machine learning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

### Load in Raw Data

The raw data is a collection of purchases from an online retailer collected in 2011. Each row in the original data represents one product that was purchased with multiple purchases forming an order. There are a few issues with the data that will need to be addressed (as with most real-world datasets)!

This code loads in the data from an s3 bucket, converts the price in Pounds to dollars (based on the exchange rate on May 31, 2011), and creates a column representing the total of the purchase. 

In [ ]:
csv_s3 = "s3://featurelabs-static/online-retail-logs.csv"
data = pd.read_csv(csv_s3, parse_dates=["order_date"])

# Convert to dollars
data['price'] = data['price'] * 1.65
data['total'] = data['price'] * data['quantity']

# Restrict data to 2011
data = data[data['order_date'].dt.year == 2011]

data.head()

## Data Cleaning

There are a few issues we need to address with the data. First, we'll drop the duplicated rows, then we'll drop any rows that contain a `nan`. Finally, we can add a `Boolean` column indicating whether or not an order is a cancellation.

In [ ]:
# drop the duplicates
data = data.drop_duplicates()

# drop rows with null customer id
data = data.dropna(axis=0)

data['cancelled'] = data['order_id'].str.startswith('C')
data['cancelled'].value_counts().plot.bar(figsize = (6, 5));
plt.title('Cancelled Purchases Breakdown');

All of the cancelled orders have negative quantities which mean that they cancel out with the corresponding purchase. We'll leave in the cancelled purchases, because if our goal (defined later in the prediction problem) is to predict the total amount purchased by a customer, we'll need to take into account their cancelled orders.

In [ ]:
data.describe()

We can see that most total purchase prices are less than \$50. We can plot the purchase total by country (limited to only positive amounts and less than \$1000) to see if there are differences between countries.

In [ ]:
plt.figure(figsize = (20, 6))
sns.boxplot(x = 'country', y = 'total', data = data[(data['total'] > 0) & (data['total'] < 1000)]);
plt.title("Total Purchase Amount by Country");
plt.xticks(rotation = 90);

In [ ]:
plt.figure(figsize = (20, 6))
sns.boxplot(x = 'country', y = 'quantity', data = data[(data['total'] > 0) & (data['total'] < 1000)]);
plt.title("Purchased Quantity by Country");
plt.xticks(rotation = 90);

Both the purchase total and the quantity are heavily skewed. This occurs often in real-world data and means this might be difficult as a regression problem (predict the actual spending amount). We might want to frame the problem as classification because the large purchase totals could throw off a machine learning algorithm. Our other option would be to remove the outliers, but given that these are probably legitimate, that does not seem like a responsible choice!

To see the extent of how skewed the data is, we can use an [Empirical Cumulative Distribution Function (ECDF) plot](https://www.itl.nist.gov/div898/software/dataplot/refman1/auxillar/ecdfplot.htm). 

In [ ]:
def ecdf(data):
    x = np.sort(data)
    y = np.arange(1, len(x) + 1) / len(x)
    return x, y

In [ ]:
plt.figure(figsize = (14, 4))

# Total
plt.subplot(121)
x, y = ecdf(data.loc[data['total'] > 0, 'total'])
plt.plot(x, y, marker = '.')
plt.xlabel('Total'); plt.ylabel('Percentile'); plt.title('ECDF of Purchase Total');

# Quantity
plt.subplot(122)
x, y = ecdf(data.loc[data['total'] > 0, 'quantity'])
plt.plot(x, y, marker = '.')
plt.xlabel('Quantity'); plt.ylabel('Percentile'); plt.title('ECDF of Purchase Quantity');

# Prediction Problem

The goal of machine learning is to predict some quantity (regression) or a label (classification). Our data exploration showed that regression might not be the best approach because of the outliers, so we'll choose classification. We'll frame the problem as predicting whether or not a customer will spend more than \$500 in the next month. This could be useful to a business because it will let them market more effectively to those customers who are likely to spend more. Moreover, an online retailer could advertise differently to customers based on their predicted class of spending. 

Instead of picking just a single month for predictions, we can use each customer as a label multiple times. In other words, we not only predict whether a given customer will spend more than \$500 in May, but we also ask the same question in June, July, and so on. The thing to note is that for each month, we _can't use data from the future_ to predict the class of spending. Each month we can use information from _any previous month_ which means that our predictions should get more accurate as we advance further in time through the data since we'll be able to use more information. Each label for a customer therefore has a different set of features because there is more or less data available to us depending on the month. Doing this by hand is very tedious and error-prone, but we'll see how Featuretools is able to handle the times associated with each label to only generate legitimate features. 

## Making Labels

The function below takes in a start date and an end date (which we can set to 30 days apart) and generates a dataframe of the labels, which depends on how much the customer spent in the period. For customers who appear in the data prior to the start date but then do have a purchase in between the start and end date, we want to set their total to 0. If we simply did not include them in the labels, then that would be cheating since we have no way of knowing ahead of time that they will not spend anything in the next month.

In [ ]:
def make_retail_cutoffs_total(start_date, end_date, threshold = 500):
    
    # Find customers who exist before start date
    customer_pool = data[data['order_date'] < start_date]['customer_id'].unique()
    tmp = pd.DataFrame({'customer_id': customer_pool})

    # For customers in the customer pool, find their sum between the start and end date
    totals = data[data['customer_id'].isin(customer_pool) & 
        (data['order_date'] > start_date) & 
        (data['order_date']<end_date)
    ].groupby('customer_id')['total'].sum().reset_index()
    
    # Merge with all the customer ids to record all customers who existed before start date
    totals = totals.merge(tmp, on = 'customer_id', how = 'right')

    # Set the total for any customer who did not have a purchase in the timeframe equal to 0
    totals['total'] = totals['total'].fillna(0)
    
    # Label is based on the threshold
    totals['label'] = (totals['total'] > threshold).astype(int)
        
    # The cutoff time is the start date
    totals['cutoff_time'] = pd.to_datetime(start_date)
    totals = totals[['customer_id', 'cutoff_time', 'total', 'label']]
    
    return totals

In [ ]:
may_spending = make_retail_cutoffs_total(pd.datetime(2011, 5, 1), pd.datetime(2011, 6, 1))
may_spending.head()

For each customer who appeared in the data before May, we have a label for them for the month of May. When we make features for these labels, we can only use data from _before May_. The `cutoff_time` represents the point at which any data we use must come before and the `label` is based on our threshold of \$500. 

In [ ]:
may_spending['label'].value_counts().plot.bar();
plt.title('Label Distribution for May');

This is an imbalanced classification problem which means that we probably don't want to use accuracy as our metric.

### Metrics

Instead of plain accuracy, we can measure performance in terms of:

* Precision: the percentage of customers predicted to spend more than \$500 that actually did
* Recall: the percentage of customers that actually spent more than \$500 that were correctly identified
* F1 score: the harmonic mean of precision and recall
* Receiver Operating Characterisic Area Under the Curve (ROC AUC): a 0 to 1 measure (with 1 being optimal) that measures the performance of a model across a range of thresholds

We'll have to establish a baseline for these metrics (which will be a little later) so we know whether machine learning is useful for this task.

Next we'll go ahead and make labels for the rest of the year. Keep in mind that this will generate one label for each customer for each month. We're used to thinking of a single label per customer, but since we have the data, we might as well use each customer as a training example as many times as possible (as long as we use valid data each time).

In [ ]:
march_spending = make_retail_cutoffs_total('2011-03-01', '2011-04-01', 500)
april_spending = make_retail_cutoffs_total('2011-04-01', '2011-05-01', 500)
june_spending = make_retail_cutoffs_total('2011-06-01', '2011-07-01', 500)
july_spending = make_retail_cutoffs_total('2011-07-01', '2011-08-01', 500)
august_spending = make_retail_cutoffs_total('2011-08-01', '2011-09-01', 500)
september_spending = make_retail_cutoffs_total('2011-09-01', '2011-10-01', 500)
october_spending = make_retail_cutoffs_total('2011-10-01', '2011-11-01', 500)
november_spending = make_retail_cutoffs_total('2011-11-01', '2011-12-01', 500)
december_spending = make_retail_cutoffs_total('2011-12-01', '2012-01-01', 500)

In [ ]:
labels = pd.concat([march_spending, april_spending, may_spending, june_spending, july_spending, august_spending, 
                    september_spending, october_spending, november_spending, december_spending], axis = 0)
labels.to_csv('../input/labels.csv')
labels.describe()

We have roughly 25,000 labels with ~17% of them positive. The total is very skewed, with several customers recording negative months (they had more cancellations than purchases). By framing this as a classification problem, we don't have to worry about the outlying purchase totals throwing off our model. 

Just to examine the data, we can plot the total spending distribution by month (with negative totals removed).

In [ ]:
plot_labels = labels.copy()
plot_labels['month'] = plot_labels['cutoff_time'].dt.month

plt.figure(figsize = (12, 6))
sns.boxplot(x = 'month', y = 'total', 
            data = plot_labels[(plot_labels['total'] > 0) & (plot_labels['total'] < 1000)]);
plt.title('Customer Spending Distribution by Month');

Let's zoom in to one customer to make sure we understand the labels. 

In [ ]:
labels.loc[labels['customer_id'] == 12347]

In [ ]:
labels.loc[labels['customer_id'] == 12347].set_index('cutoff_time')['total'].plot(figsize = (6, 4), linewidth = 3)
plt.xlabel('Date', size = 16); 
plt.ylabel('Spending', size = 16); 
plt.title('Monthly Spending for Customer', size = 20);
plt.xticks(size = 16); plt.yticks(size = 16);

One customer, 8 different labels. It seems like it might be a little difficult to predict this customer's habits given her fluctuating total spending! We'll have to see if Featuretools is up to the task.

We now have our prediction problem all set. The next step is to start making features we can use in a machine learning model.

# Featuretools Implementation

The first step in Featuretools is to create an `EntitySet` which will hold all of our data and the relationships between the multiple tables (which we'll create shortly). Initially we'll add the entire `data` as an `entity` to the set. Since data has a `time_index`, we'll add that and specify the variable type of the product description.

In [ ]:
es = ft.EntitySet(id="Online Retail Logs")

# Add the entire data table as an entity
es.entity_from_dataframe("purchases",
                         dataframe=data,
                         index="purchases_index",
                         time_index = 'order_date',
                         variable_types={'description': ft.variable_types.Text})

### Normalizing Entities

In order to create new tables out of the original table, we can normalize this `entity`. This creates new tables by creating a unique row for every variable that we pass in, such as the customer or the product.

The code below creates a new entity for the `products` where each row contains one product and the columns describe the product. 

In [ ]:
# create a new "products" entity
es.normalize_entity(new_entity_id="products",
                    base_entity_id="purchases",
                    index="product_id",
                    additional_variables=["description"])

es['products'].df.head()

The `first_purchases_time` is automatically created because the `purchases` table has a time index. This represents the first time the product appears in the purchase data. We can use this table to create new features; the products table is a parent of the `purchases` table with the linking variable `product_id`. For each product in `products`, there can be multiple purchases of that product in `purchases`. 

We'll repeat the process to create tables for both the `customers` and the `orders`. Using `normalize_entity` automatically creates the relationships and time index so we don't have to do that ourselves. If we want to include any other additional variables in the table, we can pass those in. These variables must be unique to the object that we are normalizing for. As an example, each order comes from one country and one customer, so we can include those as additional variables when creating the `orders` table. However, the description is not unique to an order, so that should not be a variable that appears in the orders table.

In [ ]:
es['purchases']

In [ ]:
# create a new "customers" entity based on the orders entity
es.normalize_entity(new_entity_id="customers",
                    base_entity_id="purchases",
                    index="customer_id")

# create a new "orders" entity
es.normalize_entity(new_entity_id="orders",
                    base_entity_id="purchases",
                    index="order_id",
                    additional_variables=["country", 'cancelled'])

es

# Deep Feature Synthesis 

Now that our `EntitySet` is defined with the proper relationships between tables, we can perform [deep feature synthesis](http://www.jmaxkanter.com/static/papers/DSAA_DSM_2015.pdf) to generate 100s or 1000s of features. We can theoretically make features for any entity, but since our objective is to classify customer spending, we'll make features for each customer for each month. 

The following call will generate features for each customer, resulting in a `feature_matrix` where each row consists of one customer for one month and each column is one feature. 

## Using cutoff times

To ensure the features are valid for the customer and the month, we'll pass in the labels dataframe that has the cutoff time for each customer for each month. Featuretools will make one row for each customer for each month, with the features for each month derived only from data prior to the cutoff time. This is an _extremely useful method_ because it means we don't have to worry about using illegal data to make features. For example, if we were doing this by hand, it would be very easy to create features that use information from the future to make features for months, which is not allowed (well there's nothing stopping us from doing this, but it will lead to very poor models in production). 

The requirements of the cutoff time dataframe are that the first column contains the ids corresponding to the index of the target entity, and the second column must have the cutoff times. Featuretools then takes care of the rest, for each month only using valid data. 

In [ ]:
feature_matrix, feature_names = ft.dfs(entityset=es, target_entity='customers',
                                       cutoff_time = labels, verbose = 2,
                                       cutoff_time_in_index = True,
                                       chunk_size = len(labels), n_jobs = -1,
                                       max_depth = 1)

feature_matrix.head()

We will want to drop the `total` and `label` columns before training because these were passed through from the `cutoff_time` data. (We clearly can't use these for modeling!) We also can remove the `MODE` of the order id and product id. These should not be used for creating features since they are index variables.

In [ ]:
feature_matrix = feature_matrix.drop(columns = ['MODE(purchases.order_id)', 'MODE(purchases.product_id)'])
feature_matrix.shape

Initially we did not generate very many features because we limited the `max_depth` to 1. This means that only 1 aggregation will be stacked at a time. Let's take a look at some of the features. First, we'll zoom back in to a single one of the customers.

In [ ]:
feature_matrix.loc[12347, :].sample(10, axis = 1)

If we want to predict spending in November, we can use any data from before November. As we get deeper into the year, we would expect our predictions to get more accurate because we are incorporating more information (we can see the count of purchases increasing as we use more data).

In [ ]:
# feature_matrix.reset_index(inplace = True)

feature_matrix.groupby('time')['COUNT(purchases)'].mean().plot();
plt.title('Average Monthly Count of Purchases');
plt.ylabel('Purchases Per Customer');

This shows that as we progress through time, we have more purchases per customer to use for prediction.

In [ ]:
feature_matrix.groupby('time')['SUM(purchases.quantity)'].mean().plot();
plt.title('Average Monthly Sum of Purchased Products');
plt.ylabel('Total Purchased Products Per Customer');

Naturally, as we include more information, our forecasts should improve in accuracy. Therefore, if we are predicting purchase in November, we would expect better performance than predicting purchases in June.

## Correlations

As a first approximation of useful features, we can see if there are any significant correlations between the features and the `total`. We'll one-hot encode the categorical features first.

In [ ]:
feature_matrix = pd.get_dummies(feature_matrix).reset_index()
feature_matrix.shape

In [ ]:
corrs = feature_matrix.corr().sort_values('total')
corrs['total'].head()

In [ ]:
corrs['total'].dropna().tail()

A few of the features have a moderate positive correlation with the `total` (ignoring the `label` for now). The number and total of the purchases is clearly related to the total spending! 

In [ ]:
g = sns.FacetGrid(feature_matrix[(feature_matrix['SUM(purchases.total)'] > 0) & (feature_matrix['SUM(purchases.total)'] < 1000)],
                  hue = 'label', size = 4, aspect = 3)
g.map(sns.kdeplot, 'SUM(purchases.total)')
g.add_legend();
plt.title('Distribution of Purchases Total by Label');

In [ ]:
feature_matrix['month'] = feature_matrix['time'].dt.month
sns.violinplot(x = 'month', y = 'NUM_UNIQUE(purchases.order_id)', hue = 'label', figsize = (24, 6),
               data = feature_matrix[(feature_matrix['SUM(purchases.total)'] > 0) & (feature_matrix['SUM(purchases.total)'] < 1000)])
plt.title('Number of Unique Purchases by Label');

## Preliminary modeling

We can now directly use this feature matrix for training and making predictions with a machine learning model. We have a single table of data, so we'll want to split it into separate training and testing sets. For each month, we can do this by first subsetting the data and then using a random split into 30% testing and 70% training. If we want to make predictions for July, we'll subset the data to months before July, then split the data into the training and testing sets. 


### Model

For a model, we will use the `RandomForestClassifier` as implemented in Scikit-Learn. We'll keep most of the hyperparameters at the default values but increase the number of trees to 1000. This is not an optimized model but should allow us to tell whether or not our solution is better than a baseline estimate.

In [ ]:
model = RandomForestClassifier(n_estimators = 1000, 
                               random_state = 50,
                               n_jobs = -1)

The function below trains and tests for a single month. We pass in the month, and the usable data is subsetted to before the month, and then split into 30% testing and 70% training. The model is trained and makes predictions (both the labels and the probabilities) which we can assess in terms of the metrics. 

In [ ]:
def predict_month(month, feature_matrix, return_probs = False):
    
    
    feature_matrix['month'] = feature_matrix['time'].dt.month
    
    test_labels = feature_matrix.loc[feature_matrix['month'] == month, 'label']
    train_labels = feature_matrix.loc[feature_matrix['month'] < month, 'label']
    
    # Features
    X_train = feature_matrix[feature_matrix['time'].dt.month < month].drop(columns = ['customer_id', 'time',
                                                                                     'month', 'label', 'total'])
    X_test = feature_matrix[feature_matrix['time'].dt.month == month].drop(columns = ['customer_id', 'time',
                                                                                     'month', 'label', 'total'])
    feature_names = list(X_train.columns)
    
    pipeline = Pipeline([('imputer', Imputer(strategy = 'median')), 
                      ('scaler', MinMaxScaler())])

    # Fit and transform training data
    X_train = pipeline.fit_transform(X_train)
    X_test = pipeline.transform(X_test)
    
    # Labels
    y_train = np.array(train_labels).reshape((-1, ))
    y_test = np.array(test_labels).reshape((-1, ))
    
    print('Training on {} observations.'.format(len(X_train)))
    print('Testing on {} observations.\n'.format(len(X_test)))
    
    # Train 
    model.fit(X_train, y_train)
    
    # Make predictions
    predictions = model.predict(X_test)
    probs = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    p = precision_score(y_test, predictions)
    r = recall_score(y_test, predictions)
    f = f1_score(y_test, predictions)
    auc = roc_auc_score(y_test, probs)
    
    print(f'Precision: {round(p, 5)}')
    print(f'Recall: {round(r, 5)}')
    print(f'F1 Score: {round(f, 5)}')
    print(f'ROC AUC: {round(auc, 5)}')
    
    # Feature importances
    fi = pd.DataFrame({'feature': feature_names, 'importance': model.feature_importances_})
    
    if return_probs:
        return fi, probs
    
    return fi
    

In [ ]:
june_fi = predict_month(6, feature_matrix)

We can plot the feature importances using a utility function. These should allow us to see what the model considers useful information for predicting spending.

In [ ]:
from utils import plot_feature_importances

norm_june_fi = plot_feature_importances(june_fi)

The most important features are those that are most correlated with the total. This should give us confidence that our machine learning model is learning the important relationships and that the Featuretools features are useful for the problem.

## Comparison to Baseline

One question we have to ask is how do those numbers compare to an informed baseline. If our model can't beat a simple baseline, then we might want to question our approach or even if machine learning is applicable to the problem. 

For an informed baseline, let's use the amount the customer spent in the past month to predict how much they will spend in the next month. We can try this for July 2011. For the probability (used for the ROC AUC), we'll divide the previous month's total by the threhold (so a spending of 0 corresponds to 0 probability) and then clip any values to 1.

In [ ]:
labels['month'] = labels['cutoff_time'].dt.month
july_labels = labels[labels['month'] == 7]
june_labels = labels[labels['month'] == 6]

july_labels = july_labels.rename(columns = {'total': 'july_total'})
june_labels = june_labels.rename(columns = {'total': 'june_total'})

# Merge the current month with the previous
july_labels = july_labels.merge(june_labels[['customer_id', 'june_total']], on = 'customer_id', how = 'left')
july_labels['june_total'] = july_labels['june_total'].fillna(0)
july_labels['predicted_label'] = (july_labels['june_total'] > 500).astype(int)

july_labels['probability'] = july_labels['june_total'] / 500
    
# Set probabilities greater than 1 equal to 1
july_labels.loc[july_labels['probability'] > 1, 'probability'] = 1
    
july_labels.sample(10, random_state=50)

To test whether this is reasonable, we can find the correlation between the previous months total and the current months total.

In [ ]:
july_labels['july_total'].corr(july_labels['june_total'])

There is a moderate correlation between spending from one month to the next.

In [ ]:
sns.lmplot('june_total', 'july_total', data = july_labels, fit_reg = False)
plt.title('July vs June Spending');

In [ ]:
print('Precision: {:.5f}.'.format(precision_score(july_labels['label'], july_labels['predicted_label'])))
print('Recall: {:.5f}.'.format(recall_score(july_labels['label'], july_labels['predicted_label'])))
print('F1 Score: {:.5f}.'.format(f1_score(july_labels['label'], july_labels['predicted_label'])))
print('ROC AUC Score: {:.5f}.'.format(roc_auc_score(july_labels['label'], july_labels['probability'])))

We can now compare this performance to that from the model. 

In [ ]:
july_fi, july_probs = predict_month(7, feature_matrix, True)

For a classifier, the most important metric is the ROC AUC because that accounts for performance across all possible thresholds. We can adjust the threshold to maximize the Recall/Precision/F1 Score depending on our preferences.

To make sure that our model is really outperforming the baseline, we can plot the Receiver Operating Characteristic Curve for the two sets of predictions.

In [ ]:
# Calculate false positive rates and true positive rates
base_fpr, base_tpr, _ = roc_curve(july_labels['label'], july_labels['probability'])
model_fpr, model_tpr, _ = roc_curve(feature_matrix[feature_matrix['month'] == 7]['label'], july_probs)

plt.figure(figsize = (8, 6))
plt.rcParams['font.size'] = 16
# Plot both curves
plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
plt.plot(model_fpr, model_tpr, 'r', label = 'model')
plt.legend();
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('ROC Curves');

Based on the metrics and this plot, we can say that our model does indeed outperform the informed baseline guess. Machine learning, and the features from Featuretools, yield us a better solution than using domain knowledge. 

__If we were an online retailer currently using an informed guess for advertising, machine learning could make our advertising campaigns more efficient.__

Let's take all of the code for comparing the baseline to the model and put it in a single function. First we need to calculate the informed baseline in a function. 

In [ ]:
def informed_baseline(month_number, threshold = 500):
    """Calculate an informed baseline for a given month. 
    The informed baseline is guessing the previous month's spending
    for the next month. The probability is assessed by dividing
    the previous month's total by the threshold and setting
    any values greater than 1 to 1."""
    
    # Subset to the months
    month = labels[labels['month'] == month_number]
    previous_month = labels[labels['month'] == (month_number - 1)]
    
    previous_month = previous_month.rename(columns = {'total': 'previous_total'})

    # Merge the current month with the previous month
    month = month.merge(previous_month[['customer_id', 'previous_total']], on = 'customer_id', how = 'left')
    
    month['previous_total'] = month['previous_total'].fillna(0)
    month['probability'] = month['previous_total'] / threshold
    
    # Set probabilities greater than 1 equal to 1
    month.loc[month['probability'] > 1, 'probability'] = 1
    
    # Make the predicted label
    month['prediction'] = (month['previous_total'] > threshold).astype(int)
    
    print('Precision: {:.5f}.'.format(precision_score(month['label'], month['prediction'])))
    print('Recall: {:.5f}.'.format(recall_score(month['label'], month['prediction'])))
    print('F1 Score: {:.5f}.'.format(f1_score(month['label'], month['prediction'])))
    print('ROC AUC Score: {:.5f}.'.format(roc_auc_score(month['label'], month['probability'])))
    
    return month

In [ ]:
def compare(month, feature_matrix):
    """Compare machine learning model to baseline performance.
    Computes statistics and shows ROC curve."""
    
    print('Baseline Performance')
    baseline = informed_baseline(month)
    
    print('\nModel Performance')
    fi, probs = predict_month(month, feature_matrix, return_probs=True)
    
    # Calculate false positive rates and true positive rates
    base_fpr, base_tpr, _ = roc_curve(baseline['label'], baseline['probability'])
    model_fpr, model_tpr, _ = roc_curve(feature_matrix[feature_matrix['month'] == month]['label'], probs)

    plt.figure(figsize = (8, 6))
    plt.rcParams['font.size'] = 16
    
    # Plot both curves
    plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
    plt.plot(model_fpr, model_tpr, 'r', label = 'model')
    plt.legend();
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('ROC Curves');
    

In [ ]:
compare(6, feature_matrix)

Let's test December, when theoretically our model should do the best because it is training on the most data.

In [ ]:
compare(12, feature_matrix)

We get a slightly better ROC AUC. It's possible that the data does not stay consistent over the couse of the year so using more data might not necessarily create a better model. In other words, consumer behavior can shift (due to seasonality) so the older data might not be relevant.

# Tuning Deep Feature Synthesis

We saw that even a basic implementation of Featuretools can yield a set of useful features. Next, we want to ask if we can achieve even better performance by generating a richer set of features. We can expand the capabilities of Featuretools by specifying the primitives we use, increasing the maximum depth of stacked features, writing our own custom primitives, identifying interesting values, and using seed features. For an explanation of all these concepts, refer to the [Featuretools documentation.](https://docs.featuretools.com/guides/tuning_dfs.html)

In the code cell below, we again use Featuretools but this time specify a few more primitives and increase the maximum depth. This might create some unnecessary features, but we can then use feature selection to remove them. Having too many features is a better problem than having too few! 

In [ ]:
feature_matrix, feature_names = ft.dfs(entityset=es, target_entity='customers',
                                       agg_primitives = ['std', 'max', 'min', 'mode',
                                                         'mean', 'skew', 'last', 'avg_time_between'],
                                       trans_primitives = ['cum_sum', 'cum_mean', 'day', 
                                                           'month', 'hour', 'weekend'],
                                       n_jobs = -1, chunk_size = len(labels), max_depth = 2,
                                       cutoff_time = labels,
                                       verbose = 1)
feature_matrix.head()

Before we can use these features for modeling, we need to one-hot encode them. Because some of the features are built from the id variables, one-hot encoding them will create far too many features. We'll have to be careful about increasing the number of columns too greatly.

It's interesting that the most important feature is whether the most common country for the order is the United Kingdom. This suggests there may a difference in order that come from the United Kingdom versus orders from other countries.

In [ ]:
orders = es['orders'].df
order_products = es['order_products'].df
order_products = order_products.merge(orders[['order_id', 'country']], 
                                      on = 'order_id', how = 'left')

orders_by_country = order_products.groupby('country')['total'].agg(['sum', 'mean', 'count'])
orders_by_country.head()

In [ ]:
orders_by_country['count'].sort_values().plot.bar(color = 'green', 
                                    figsize = (10, 8));
plt.title('Number of Orders by Country');

In [ ]:
orders_by_country['mean'].sort_values().plot.bar(color = 'red', 
                                    figsize = (12, 8));
plt.title('Mean of Order Total by Country', size = 22);
plt.ylabel('Mean Order Total ($)', size = 18); plt.xlabel('Country', size = 16)
plt.xticks(size = 16); plt.yticks(size = 16);

Let's try some minor feature selection and see if the performance improves. The following call 
removes:

* One out of every pair of columns that have a perfect correlation (all values are equal)
* Any features with more than 80% missing values 
* Any features with only a single unique value
* One out of every pair of columns with a correlation greater than 0.9.

In [ ]:
t = feature_matrix_enc['time']
feature_matrix_selected = feature_selection(feature_matrix_enc.drop(columns = ['time']),
                                                                    missing_threshold=80, correlation_threshold=0.9)
feature_matrix_selected['time'] = t

## Custom Primitives

We saw that an informed baseline guess just using the previous month's total yielded a decent solution (although worse than machine learning). Therefore, we might want to consider using this as a feature in our model. To build this feature (which will actually find the total of any numeric column for the previous month) we can write a custom primitive.

In [ ]:
ct = pd.datetime(2011, 6, 1)
ct.month

In [ ]:
from featuretools.primitives import make_agg_primitive

In [ ]:
def total_previous_month(numeric, datetime, time):
    df = pd.DataFrame({'value': numeric, 'time': datetime})
    previous_month = time.month - 1
    
    df = df[df['time'].dt.month == previous_month]
    total = df['value'].sum()
    
    return total

In [ ]:
total_previous = make_agg_primitive(total_previous_month, input_types = [ft.variable_types.Numeric,
                                                                         ft.variable_types.Datetime],
                                    return_type = ft.variable_types.Numeric, 
                                    uses_calc_time = True)

In [ ]:
feature_matrix, feature_names = ft.dfs(entityset=es, target_entity='customers',
                                       cutoff_time = labels, 
                                       cutoff_time_in_index = True, 
                                       agg_primitives = [total_previous], 
                                       trans_primitives = [], chunk_size = 100,
                                       verbose = 1, n_jobs = 4)

In [ ]:
feature_matrix.drop_duplicates(inplace = True)
feature_matrix.head()

In [ ]:
feature_matrix.shape

In [ ]:
labels

In [ ]:
feature_matrix.head()

In [ ]:
december_fi = predict_month(12, feature_matrix_selected)

Feature selection slightly improved the models performance compared to the original metrics (shown below):

    Precision: 0.27073
    Recall: 0.50314
    F1 Score: 0.35204
    ROC AUC: 0.86048

### Feature Selection Based on Feature Importances

Another method we can use for feature selection is to remove any features that have zero importance. In a decision tree based model, removing these features will have no effect on performance and is hence a safe option for feature selection.

In [ ]:
zero_features = december_fi.loc[december_fi['importance'] == 0, 'feature']
print(f'{len(zero_features)} features with no importance.')

In [ ]:
feature_matrix_selected = feature_matrix_selected[[x for x in feature_matrix_selected if x not in zero_features]]
feature_matrix_selected.shape

## Tuning Deep Feature Synthesis

We can specify different primitives to deep feature synthesis in order to generate different features. Let's try making a new feature matrix with specified primitives.

After building the new feature matrix, we need to one-hot encode it. We can also use feature selection because that improves performance.

In [ ]:
feature_matrix_enc = pd.get_dummies(feature_matrix)

for col in feature_matrix_enc:
    if feature_matrix_enc[col].dtype == 'bool':
        feature_matrix_enc[col] = feature_matrix_enc[col].astype(np.uint8)

feature_matrix_selected = feature_selection(feature_matrix_enc)

In [ ]:
feature_matrix_selected.reset_index(inplace = True)

# Add the time
feature_matrix_selected['time'] = list(labels['cutoff_time'])

june_fi = predict_month(6, feature_matrix_enc)

The previous metrics for June were:

    Precision: 0.51842
    Recall: 0.46244
    F1 Score: 0.48883
    ROC AUC: 0.78358

In [ ]:
norm_june_fi = plot_feature_importances(june_fi)

In [ ]:
december_fi = predict_month(12, feature_matrix_enc)

In [ ]:
_ = informed_baseline(12)

The model with more features from Featuretools again significantly outperforms the baseline. To improve the precision, recall, and consequently the F1 score, we can tune the threshold of predictions made by the model.

## Precision Recall Curve

We can use the precision recall curve to identify the ideal threshold for our model. For example, we might care more about identifying the customers likely to spend more 

In [ ]:
month = 12
# Find the probability
fi, probs = predict_month(month, feature_matrix_selected, 
                          return_probs = True)

precision, recall, t = precision_recall_curve(labels.loc[labels['cutoff_time'].dt.month == month, 'label'], 
                                              probs)

plt.step(recall, precision, color='b', alpha=0.5,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.5,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title("Precision vs. Recall Curve"); plt.show();

In [ ]:
plt.text(precision[::10], recall[::10], s = )

In [ ]:
def precision_recall(month):
    "Show the precision vs recall curve for a month"
    
    # Find the probability
    fi, probs = predict_month(month, feature_matrix_selected, 
                              return_probs = True)
    
    precision, recall, t = precision_recall_curve(labels.loc[labels['cutoff_time'].dt.month == month, 'label'], 
                                                  probs)
    
    plt.step(recall, precision, color='b', alpha=0.5,
             where='post')
    plt.fill_between(recall, precision, step='post', alpha=0.5,
                     color='b')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title("Precision vs. Recall Curve"); plt.show();

In [ ]:
precision_recall(12)


We can use this plot to select an appropriate threshold. For example, we may care more about identifying customers who will spend more than \$500 than about false positive so we choose a threshold that results in high recall but low precision. Conversely, we may want to limit false positives even if that means missing some potential high-spending customers so we would try for a higher level of precision.

In [ ]:
t